In [ ]:
!pip install keras-tuner
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense, Dropout)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

def build_model(hp):
    model = Sequential()
    
    # Hyperparameters to tune
    hp_filters = hp.Choice('filters', values=[32, 64, 128])
    hp_dropout_conv = hp.Float('dropout_conv', min_value=0.1, max_value=0.4, step=0.1)
    hp_dropout_dense = hp.Float('dropout_dense', min_value=0.3, max_value=0.5, step=0.1)
    hp_units_dense1 = hp.Int('units_dense1', min_value=256, max_value=1024, step=128)
    hp_units_dense2 = hp.Int('units_dense2', min_value=128, max_value=512, step=64)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    # First Block
    model.add(Conv2D(hp_filters, (3, 3), padding='same', activation='relu', input_shape=(48, 48, 1)))
    model.add(Conv2D(hp_filters, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(hp_dropout_conv))
    
    # Second Block
    model.add(Conv2D(hp_filters * 2, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(hp_filters * 2, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(hp_dropout_conv))
    
    # Third Block
    model.add(Conv2D(hp_filters * 4, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(hp_filters * 4, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(hp_dropout_conv))
    
    # Fourth Block
    model.add(Conv2D(hp_filters * 8, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(hp_filters * 8, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(hp_dropout_conv))
    
    model.add(Flatten())
    
    # Fully Connected Layers
    model.add(Dense(hp_units_dense1, activation='relu'))
    model.add(Dropout(hp_dropout_dense))
    
    model.add(Dense(hp_units_dense2, activation='relu'))
    model.add(Dropout(hp_dropout_dense))
    
    model.add(Dense(len(emotion_map), activation='softmax'))
    
    # Compile the model
    optimizer = Adam(learning_rate=hp_learning_rate)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Set up the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=1,
    directory='my_dir',
    project_name='emotion_recognition_tuning'
)

# Callbacks
checkpoint = ModelCheckpoint(
    filepath='best_model.keras',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-5,
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

callbacks = [checkpoint, reduce_lr, early_stopping]

# Run the hyperparameter search
tuner.search(
    train_generator,
    steps_per_epoch=len(train_data) // BATCH_SIZE,
    epochs=25,
    validation_data=val_generator,
    validation_steps=len(val_data) // BATCH_SIZE,
    callbacks=callbacks
)

# Retrieve the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)

# Train the best model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_data) // BATCH_SIZE,
    epochs=25,
    validation_data=val_generator,
    validation_steps=len(val_data) // BATCH_SIZE,
    callbacks=callbacks
)

# Save the best model
model.save('best_emotion_model.h5')


Trial 17 Complete [00h 10m 31s]
val_accuracy: 0.2539626657962799

Best val_accuracy So Far: 0.6107784509658813
Total elapsed time: 02h 19m 22s

Search: Running Trial #18

Value             |Best Value So Far |Hyperparameter
32                |64                |filters
0.5               |0.2               |dropout_conv
0.5               |0.2               |dropout_dense
1024              |512               |units_dense1
448               |320               |units_dense2
0.0001            |0.001             |learning_rate